In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import plotly.express as px

In [4]:
# Leer los CSVs
nivel_real = pd.read_csv('processed/puertos_estado_real_datos.csv')
shom = pd.read_csv('processed/shom_datos_2024-10-12_06-00.csv')
puertos = pd.read_csv('processed/puertos_estado_datos_2024-10-12_06-00.csv')
armada = pd.read_csv('processed/armada_datos_2024-10-12_06-00.csv')
copernicus = pd.read_csv('processed/copernicus_datos_2024-10-12_06-00.csv')

# Convertir las columnas de fechas a formato datetime
nivel_real['Fecha'] = pd.to_datetime(nivel_real['Fecha'])
shom['Fecha'] = pd.to_datetime(shom['Fecha'])
puertos['Fecha'] = pd.to_datetime(puertos['Fecha'])
armada['Fecha'] = pd.to_datetime(armada['Fecha'])
copernicus['Fecha'] = pd.to_datetime(copernicus['Fecha'])

# 1. Calcular el Z-score para la columna 'Nivel Medio'
z_scores = stats.zscore(nivel_real['Nivel Medio'])

# Definir un umbral para el Z-score (por ejemplo, 3)
umbral = 3

# Filtrar el DataFrame para eliminar outliers
nivel_real_filtrado = nivel_real[np.abs(z_scores) < umbral]

# Crear el gráfico vacío
fig = px.line(title='Previsiones marea')

# Añadir la primera línea (REDMAR) al gráfico
fig.add_scatter(x=nivel_real_filtrado['Fecha'], y=nivel_real_filtrado['Nivel Medio'], 
                mode='lines', name='REDMAR', line=dict(color='orange'))

# Añadir las otras líneas al gráfico
fig.add_scatter(x=shom['Fecha'], y=shom['Nivel'], mode='lines', name='HYCOM', line=dict(color='blue'))
fig.add_scatter(x=copernicus['Fecha'], y=copernicus['Nivel'], mode='lines', name='CMEMS', line=dict(color='green'))
fig.add_scatter(x=armada['Fecha'], y=armada['Nivel'], mode='lines', name='IHM', line=dict(color='red'))
fig.add_scatter(x=puertos['Fecha'], y=puertos['Nivel'], mode='lines', name='HAMSOM', line=dict(color='purple'))

# Configurar el rango visible en el eje X (opcional)
start_date = '2024-10-12 00:00:00'
end_date = '2024-10-15 00:00:00'
fig.update_xaxes(range=[start_date, end_date])

# Configurar el diseño del gráfico
fig.update_layout(xaxis_title='Fecha', yaxis_title='Nivel', showlegend=True)
fig.update_traces(mode='lines')

# Mostrar el gráfico
fig.show()
